# Shared Evaluation (Base + LoRA)

Evaluates base checkpoint and all LoRA adapters from one shared config.


In [1]:
from __future__ import annotations

import subprocess
import sys
from pathlib import Path


def find_repo_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if (candidate / '.git').exists():
            return candidate
    return start


REPO_ROOT = find_repo_root(Path.cwd().resolve())
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

EVAL_CONFIG_PATH = REPO_ROOT / 'configs/eval/shared_eval.yaml'
EVAL_MODE = 'quick'  # quick | full
EVAL_CONFIG_PATH


PosixPath('/teamspace/studios/this_studio/Hot-Peppers-Company-Computer-Vision/configs/eval/shared_eval.yaml')

In [2]:
from pipelines.dependencies import check_packages, assert_required_packages

status = check_packages(['torch', 'torchvision', 'yaml', 'safetensors', 'einops', 'tqdm', 'pandas'])
status


{'torch': True,
 'torchvision': True,
 'yaml': True,
 'safetensors': True,
 'einops': True,
 'tqdm': True,
 'pandas': True}

In [3]:
assert_required_packages(['torch', 'torchvision', 'yaml', 'safetensors', 'einops'])
print('Core dependencies look good')


Core dependencies look good


In [4]:
from pipelines.contracts import EvalConfig

cfg = EvalConfig.from_yaml(EVAL_CONFIG_PATH).eval
cfg


{'output_dir': 'results/shared_eval',
 'device': 'cuda',
 'batch_size': 8,
 'num_workers': 4,
 'conf_threshold': 0.25,
 'nms_iou': 0.5,
 'max_images_quick': 512,
 'save_predictions': True,
 'metrics': ['map_50',
  'map_50_95',
  'fps',
  'frames_per_watt',
  'avg_power_w',
  'energy_per_frame_j'],
 'power': {'enabled': True,
  'backend': 'auto',
  'gpu_index': 0,
  'poll_interval_ms': 100,
  'warmup_batches': 20,
  'telemetry': True,
  'telemetry_dir': 'results/shared_eval/telemetry'},
 'datasets': [{'name': 'coco_val',
   'manifest': 'configs/manifests/coco_val.json',
   'image_size': 640},
  {'name': 'bdd_day_val',
   'manifest': 'configs/manifests/bdd_day_val.json',
   'image_size': 640},
  {'name': 'bdd_night_val',
   'manifest': 'configs/manifests/bdd_night_val.json',
   'image_size': 640},
  {'name': 'acdc_val',
   'manifest': 'configs/manifests/acdc_val.json',
   'image_size': 640}],
 'models': [{'name': 'base_coco',
   'model': {'model_file': 'mamba-vision-ours/model.py',
    '

In [5]:
for dataset_cfg in cfg['datasets']:
    manifest_path = REPO_ROOT / dataset_cfg['manifest']
    if not manifest_path.exists():
        raise FileNotFoundError(f'Missing manifest: {manifest_path}')
print('All manifests found')


FileNotFoundError: Missing manifest: /teamspace/studios/this_studio/Hot-Peppers-Company-Computer-Vision/configs/manifests/bdd_day_val.json

In [6]:
cmd = [
    sys.executable,
    str(REPO_ROOT / 'scripts/pipelines/eval_from_config.py'),
    '--config',
    str(EVAL_CONFIG_PATH),
    '--mode',
    EVAL_MODE,
]
print('Running:', ' '.join(cmd))
subprocess.run(cmd, check=True)


Running: /home/zeus/miniconda3/envs/cloudspace/bin/python /teamspace/studios/this_studio/Hot-Peppers-Company-Computer-Vision/scripts/pipelines/eval_from_config.py --config /teamspace/studios/this_studio/Hot-Peppers-Company-Computer-Vision/configs/eval/shared_eval.yaml --mode quick


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Encountered more than 100 detections in a single image. This means that certain detections with the lowest scores will be ignored, that may have an undesirable impact on performance. Please consider adjusting the `max_detection_threshold` to suit your use case. To disable this warning

CalledProcessError: Command '['/home/zeus/miniconda3/envs/cloudspace/bin/python', '/teamspace/studios/this_studio/Hot-Peppers-Company-Computer-Vision/scripts/pipelines/eval_from_config.py', '--config', '/teamspace/studios/this_studio/Hot-Peppers-Company-Computer-Vision/configs/eval/shared_eval.yaml', '--mode', 'quick']' returned non-zero exit status 1.

In [7]:
import pandas as pd

metrics_dir = REPO_ROOT / cfg.get('output_dir', 'results/shared_eval')
metrics_csv = metrics_dir / 'metrics.csv'
metrics_json = metrics_dir / 'metrics.json'

print('Metrics CSV :', metrics_csv)
print('Metrics JSON:', metrics_json)

if metrics_csv.exists():
    df = pd.read_csv(metrics_csv)
    display(df)
else:
    print('metrics.csv not found')


Metrics CSV : /teamspace/studios/this_studio/Hot-Peppers-Company-Computer-Vision/results/shared_eval/metrics.csv
Metrics JSON: /teamspace/studios/this_studio/Hot-Peppers-Company-Computer-Vision/results/shared_eval/metrics.json
metrics.csv not found


## Done checklist

- [ ] All dataset manifests exist
- [ ] Quick eval completed without exception
- [ ] metrics.json and metrics.csv were generated
- [ ] Full eval run completed after all training jobs
